#### Connect the notebook to your Google Drive

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
root_dir = "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/"
gnn_root_dir = "gdrive/My Drive/Colab Notebooks/cv3dst_gnn_exercise/"

#### Some util functions

In [3]:
import sys
import os
sys.path.append(os.path.join(gnn_root_dir, 'src2'))
from tracker.object_detector import FRCNN_FPN
import torch
from PIL import Image


def crop_imgs(img, boxes, transforms):
    assert isinstance(img, Image.Image)

    crops = []  
    for box in boxes:
        box_crop = img.crop(box)
        crops.append(transforms(box_crop))
    #return np.stack(crops)
    return torch.stack(crops)


def load_obj_detect(obj_detect_model_file):
    obj_detect_nms_thresh = 0.3

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # object detector
    obj_detect = FRCNN_FPN(num_classes=2, nms_thresh=obj_detect_nms_thresh)
    obj_detect_state_dict = torch.load(obj_detect_model_file,
                                    map_location=lambda storage, loc: storage)
    obj_detect.load_state_dict(obj_detect_state_dict)
    obj_detect.eval()
    obj_detect.cuda()

    return obj_detect

## Load object detector, ReID model and transforms

In [6]:
! wget https://vision.in.tum.de/webshare/u/brasoand/cv3dst/resnet50_reid.pth

--2021-07-02 16:18:47--  https://vision.in.tum.de/webshare/u/brasoand/cv3dst/resnet50_reid.pth
Resolving vision.in.tum.de (vision.in.tum.de)... 131.159.19.110, 2a09:80c0:18::1110
Connecting to vision.in.tum.de (vision.in.tum.de)|131.159.19.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86824473 (83M)
Saving to: ‘resnet50_reid.pth’

resnet50_reid.pth   100%[===================>]  82.80M  18.1MB/s    in 5.5s    

2021-07-02 16:18:53 (15.0 MB/s) - ‘resnet50_reid.pth’ saved [86824473/86824473]



In [7]:
!ls

gdrive	resnet50_reid.pth  sample_data


In [8]:
from torchvision import transforms as T
from market.models import build_model

obj_detect= load_obj_detect(os.path.join(root_dir, 'models', 'faster_rcnn_fpn.model'))
crop_size = (256, 128)
tforms = T.Compose((T.Resize(crop_size), T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])))
reid_model = build_model('resnet34', 751, loss='softmax', pretrained=True)


reid_ckpt = torch.load('resnet50_reid.pth')
reid_model.load_state_dict(reid_ckpt)



<All keys matched successfully>

## Store all data

In [12]:
!ls 'gdrive/My Drive/Colab Notebooks/cv3dst_gnn_exercise/src2'

gnn  market  tracker


In [13]:
from tracker.data_track import MOT16Sequences
import os.path as osp
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm

stored_data_filename = 'preprocessed_data_{}_.pth'

reid_model.eval().cuda()
obj_detect.eval().cuda()

for train_test in ('train', 'test'):
    db = {}
    sequences = MOT16Sequences(f'MOT16-{train_test}', osp.join(root_dir,'data/MOT16'), vis_threshold=0.)

    for seq in sequences:
        print(f"Processing sequence {seq}")
        data_loader = DataLoader(seq, batch_size=1, shuffle=False)
        db[str(seq)]= []
        for frame_num, frame in tqdm(enumerate(data_loader)):                
            with torch.no_grad():            
                img_path = frame['img_path'][0]
                img = Image.open(img_path)

                assert frame_num + 1 == int(img_path.split('/')[-1].split('.')[0])

                # Store detected boxes
                boxes, scores = obj_detect.detect(frame['img'])

                # Store ReID embeddings
                det_crops = crop_imgs(img, boxes.cpu().numpy(), tforms)
                det_reid = reid_model(det_crops.cuda())

                det = {'boxes': boxes.cpu(),
                      'scores': scores.cpu(),
                      'reid': det_reid.cpu()}


                # Store ground truth box info        
                if 'gt' in frame and len(frame['gt']) > 0:
                    gt_ids = torch.as_tensor(list(frame['gt'].keys()))
                    gt_boxes = torch.cat([frame['gt'][id_.item()] for id_ in gt_ids], dim=0)
                    gt_vis = torch.cat([frame['vis'][id_.item()] for id_ in gt_ids], dim=0)

                    gt_crops = crop_imgs(img, gt_boxes.cpu().numpy(), tforms)
                    gt_reid = reid_model(gt_crops.cuda())

                    gt = {'ids': gt_ids.cpu(),
                          'boxes': gt_boxes.cpu(),
                          'vis': gt_vis.cpu(),
                          'reid': gt_reid.cpu()}
                else:
                    gt = None

                db[str(seq)].append({'det': det, 'gt': gt})
        assert len(db[str(seq)]) == len(data_loader)

    torch.save(db, osp.join(gnn_root_dir, 'data', stored_data_filename.format(train_test)))


Processing sequence MOT16-13


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: ignored